In [ ]:
# Análise Exploratória: Centróide de Geolocalização por CEP

# Este notebook gera uma tabela derivada com agregação de coordenadas por CEP
# para uso em modelagem dimensional (Star Schema) quando apropriado.

import pandas as pd
import numpy as np
from pathlib import Path


# 1. CARREGAMENTO DE DADOS

# Caminho para dados limpos (output do pipeline)
PASTA_TRUSTED = Path(r"C:\Users\isado\OneDrive\Área de Trabalho\projeto_etl\Trusted")
PASTA_DERIVED = PASTA_TRUSTED.parent / "Derived"

# Criar pasta para dados derivados se não existir
PASTA_DERIVED.mkdir(exist_ok=True)

# Carregar dados de geolocalização limpos
geo_full = pd.read_parquet(PASTA_TRUSTED / "clean_olist_geolocation_dataset.parquet")

print("=" * 70)
print("ANÁLISE EXPLORATÓRIA: GEOLOCALIZAÇÃO")
print("=" * 70)
print(f"\nDataset Original (Limpo):")
print(f"   Linhas: {len(geo_full):,}")
print(f"   Colunas: {geo_full.columns.tolist()}")
print(f"   Tamanho em memória: {geo_full.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


# 2. ANÁLISE EXPLORATÓRIA


print(f"\nAnálise de CEPs:")
total_ceps = geo_full['geolocation_zip_code_prefix'].nunique()
print(f"   CEPs Únicos: {total_ceps:,}")
print(f"   Média de registros por CEP: {len(geo_full) / total_ceps:.2f}")
print(f"   CEP com mais registros: {geo_full['geolocation_zip_code_prefix'].value_counts().iloc[0]}")

print(f"\nAnálise de Cidades:")
total_cidades = geo_full['geolocation_city'].nunique()
print(f"   Cidades Únicas: {total_cidades:,}")
print(f"\nTop 10 Cidades por Registros:")
print(geo_full['geolocation_city'].value_counts().head(10))

print(f"\n Análise de Estados:")
print(geo_full['geolocation_state'].value_counts())


# 3. GERAÇÃO DA DIMENSÃO CENTRÓIDE


print(f"\n" + "=" * 70)
print("GERANDO DIMENSÃO: CENTRÓIDE POR CEP")
print("=" * 70)

# Agregação por CEP: calcula centróide e pega valores únicos
geo_centroide = geo_full.groupby('geolocation_zip_code_prefix').agg({
    'geolocation_lat': 'mean',
    'geolocation_lng': 'mean',
    'geolocation_city': 'first',
    'geolocation_state': 'first'
}).reset_index()

# Adicionar contagem de pontos originais por CEP (para rastreabilidade)
contagem = geo_full.groupby('geolocation_zip_code_prefix').size().reset_index(name='qty_pontos_originais')
geo_centroide = geo_centroide.merge(contagem, on='geolocation_zip_code_prefix')

# Renomear colunas para clareza
geo_centroide.columns = [
    'geolocation_zip_code_prefix',
    'centroide_lat',
    'centroide_lng',
    'geolocation_city',
    'geolocation_state',
    'qty_pontos_originais'
]

print(f"\n Dimensão Centróide Criada:")
print(f"   Linhas: {len(geo_centroide):,} (uma por CEP único)")
print(f"   Tamanho em memória: {geo_centroide.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"   Redução de tamanho: {(1 - len(geo_centroide) / len(geo_full)) * 100:.1f}%")

print(f"\nPreview (primeiras 10 linhas):")
print(geo_centroide.head(10))


# 4. VALIDAÇÃO DA AGREGAÇÃO


print(f"\n" + "=" * 70)
print("VALIDAÇÃO DA AGREGAÇÃO")
print("=" * 70)

# Verificar valores nulos
print(f"\n Valores Nulos:")
print(geo_centroide.isnull().sum())

# Estatísticas de distribuição
print(f"\nDistribuição de Pontos Originais por CEP:")
print(geo_centroide['qty_pontos_originais'].describe())

print(f"\nCEPs com mais pontos de coleta:")
print(geo_centroide.nlargest(10, 'qty_pontos_originais')[
    ['geolocation_zip_code_prefix', 'geolocation_city', 'qty_pontos_originais']
])


# 5. SALVAMENTO DA DIMENSÃO DERIVADA

output_path = PASTA_DERIVED / "dim_geolocation_centroide.parquet"

geo_centroide.to_parquet(output_path, index=False, compression='snappy')

print(f"\n" + "=" * 70)
print(f"SALVO COM SUCESSO!")
print("=" * 70)
print(f"Caminho: {output_path}")
print(f"Total de linhas: {len(geo_centroide):,}")
print(f"\nUso Recomendado:")
print(f"   dim_geolocation = pd.read_parquet('{output_path.name}')")
print(f"   df_vendas = df_vendas.merge(dim_geolocation, on='geolocation_zip_code_prefix')")